In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


このノートブックは、Unsloth というライブラリの使い方を説明するチュートリアルです。Unsloth は、Google Colab 上で大きな言語モデル (LLM) を効率的に fine-tuning するためのツールです。オリジナルは[こちら](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Phi_4-Conversational.ipynb)

特に4bit QLoRAは、4ビット量子化によりモデルを軽量化し、その上でLoRAを用いて低ランクのアダプタのみを学習することで、通常の高精度ファインチューニングと同等の性能を、はるかに少ないリソースで実現する技術です。これにより、個人ユーザーでも限られたGPU環境で大規模言語モデルの微調整が可能となります。

# 学習内容
- このノートブックでは、以下の内容を学ぶことができます。

## データの準備:
- Fine-tuning 用のデータセットを準備する方法。ここでは、Phi-4 というフォーマットで会話を表現し、[FineTome-100k データセット](https://huggingface.co/datasets/mlabonne/FineTome-100k)を使用しています。対話形式のインストラクションファインチューニング向けの大規模なデータセットです。

## モデルのトレーニング:
- LoRA (Low-Rank Adaptation) を使用して、モデルを効率的に fine-tuning する方法。Huggingface TRL の SFTTrainer を使用し、train_on_completions メソッドでアシスタントの出力のみをトレーニングします。
    - TRL（Transformers Reinforcement Learning）は大規模言語モデルをよりユーザに沿った形に調整するための「強化学習」側面にフォーカスしたツールキットと言えます。
    - SFTTrainerはTRLライブラリ内で提供される教師あり微調整（Supervised Fine-Tuning, SFT）のためのトレーナークラスです。このクラスは、事前学習済みの言語モデルをユーザーが用意した指示応答データ（例：プロンプトと正解のペア）で微調整する際に利用されます。
    - 通常のSFTでは、プロンプトと正解応答のペアを用いて損失（例：クロスエントロピー）を計算します。train_on_completionsは、既に生成されたcompletionデータを入力として受け取り、これらのcompletionとターゲット（正解）との間で損失を計算し、モデルパラメータの更新を行う処理を実装しています。
    - 一般的に「completion」とは、モデルがプロンプトに対して生成したテキストのことを指します。chatの回答もその一つです。

## 推論:
- Fine-tuning されたモデルを使って推論を実行する方法。min_p と temperature を設定してテキストを生成したり、TextStreamer を使用して連続的な推論を行うことができます。

## モデルの保存:
- Fine-tuning されたモデルを LoRA アダプターとして保存する方法。Huggingface の push_to_hub (公開) または save_pretrained (ローカル保存) を使用します。また、float16 や GGUF 形式で保存する方法も説明されています。

## 要約
- このノートブックは、Unsloth を使用して Google Colab 上で LLM を fine-tuning し、推論を実行し、モデルを保存する方法を学ぶためのチュートリアルです。

### その他の情報
- さらに、ノートブックでは以下のような情報も提供されています。

    - Unsloth のインストール方法
    - 使用できる事前量子化モデルのリスト
    - LoRA アダプターの設定
    - ShareGPT スタイルのデータセットを HuggingFace の標準フォーマットに変換する方法
    - メモリ使用量とトレーニング時間の統計
    - VLLM や llama.cpp で使用するためのモデルの保存方法
    - Unsloth の Discord チャンネルへのリンク

### 以下は各用語の概要です。
- GGUF
  -  LLMの量子化モデルを保存するための新しいファイルフォーマットです。
  -  従来のGGMLフォーマットを進化させ、より柔軟で拡張性の高いメタデータや効率的な読み込みを実現することを目的としています。
- vLLM
  -  大規模言語モデル（LLM）の推論を高速化し、モデルのサービングを簡単かつ効率的に行えるようにするためのオープンソースライブラリです。PagedAttentionを活用しており、HuggingFace Transformersと比較して、モデルアーキテクチャの変更を一切必要とせずに、最大24倍の処理能力を実現します。
- bfloat16
    - 米Google（グーグル）が独自に定義したディープラーニング向けの浮動小数点フォーマット。 同社のディープラーニングアクセラレーター「TPU」で採用されている形式。

これらを実行するには、「Runtime」を押して、空いているTesla T4 Google Colabインスタンスで「Run all」を押してください！

Unslothを自分のコンピューターにインストールするには、[こちら](https://docs.unsloth.ai/get-started/installing-+-updating)のGithubページのインストール手順に従ってください。

### News

**推論モデルのトレーニング方法については、[当社のブログ記事](https://unsloth.ai/blog/r1-reasoning)をご覧ください。**\
すべての[モデルアップロード](https://docs.unsloth.ai/get-started/all-our-models)および[ノートブック](https://docs.unsloth.ai/get-started/unsloth-notebooks)については、当社のドキュメントをご覧ください。


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    "unsloth/gemma-2-9b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"  # Qwen 2.5 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

メモ：

| モデル名                                        | 日本語理解・生成         | 訓練データ          | 使用例      | 対応度       |
|-------------------------------------------------|--------------------------|---------------------|-------------|--------------|
| unsloth/Meta-Llama-3.1-8B-bnb-4bit              | 限定的                   | 可能性あり          | 評価未報告  | 限定的       |
| unsloth/Mistral-Small-Instruct-2409             | 良好                     | 多言語対応          | 良好        | 良好         |
| unsloth/Phi-4                                   | ほぼ不可                 | 英語中心            | 評価なし    | 非対応       |
| unsloth/Phi-4-unsloth-bnb-4bit                  | ほぼ不可                 | 英語中心            | 評価なし    | 非対応       |
| unsloth/gemma-2-9b-bnb-4bit                     | ほぼ不可                 | 英語中心            | 評価なし    | 非対応       |
| unsloth/Qwen2.5-7B-Instruct-bnb-4bit            | 良好                     | 多言語対応          | 良好        | 良好         |
| unsloth/Llama-3.2-1B-bnb-4bit                   | 限定的                   | 少量含む可能性      | 評価未報告  | 限定的       |
| unsloth/Llama-3.2-1B-Instruct-bnb-4bit          | 限定的                   | 公式外              | 評価なし    | 限定的       |
| unsloth/Llama-3.2-3B-bnb-4bit                   | 限定的                   | 少量含む可能性      | 評価なし    | 限定的       |
| unsloth/Llama-3.2-3B-Instruct-bnb-4bit          | 限定的                   | 公式外              | 評価なし    | 限定的       |

パラメータ効率の微調整を行うために、LoRAアダプターを追加しました。これにより、すべてのパラメータの1%のみを効率的にトレーニングできるようになります。

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### データ準備
現在、会話形式のファインチューニングには Phi-4 フォーマットを使用しています。[Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k)  データセットを ShareGPT スタイルで利用していますが、("from", "value") ではなく、HuggingFace の通常のマルチターン形式である ("role", "content") に変換しています。Phi-4 は以下のようにマルチターンの会話をレンダリングします:

```
<|im_start|>user<|im_sep|>Hello!<|im_end|>
<|im_start|>assistant<|im_sep|>Hi! How can I help?<|im_end|>
<|im_start|>user<|im_sep|>What is 2+2?<|im_end|>
```

正しいチャットテンプレートを取得するために、`get_chat_template` 関数を使用しています。 `zephyr、chatml、mistral、llama、alpaca、vicuna、vicuna_old、phi3、phi4、llama3` などに対応しています。

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
) # "phi-4" というチャットテンプレートに基づいて tokenizer を設定します。これにより、tokenizer は Phi-4 フォーマットの会話を処理できるようになります。

def formatting_prompts_func(examples):
    """この関数は、データセットの各サンプル（examples）に対して、"conversations" カラムから会話を抽出し、tokenizer.apply_chat_template を使用して Phi-4 フォーマットのテキストに変換します。そして、変換されたテキストを "text" というキーを持つ辞書として返します。
    """
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]

    return { "text" : texts, }
# pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

次に、`standardize_sharegpt`を使用して、ShareGPTスタイルのデータセットをHuggingFaceの汎用フォーマットに変換します。これにより、データセットは次のようになります。

```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

その後、先程のformatting_prompts_funcを適用してtextキーにtext valueを入れます。

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
"""formatting_prompts_func をデータセットの各サンプルに適用します。 batched=True を指定することで、処理がバッチ化され、効率的に実行されます。この処理によって、各サンプルに "text" という新しいカラムが追加され、Phi-4 フォーマットのテキストが格納されます。
"""

項目5の会話がどのように構成されているかを見てみましょう。

In [ ]:
dataset[5]["conversations"]

[{'content': '天文学者は、ドップラー効果を利用して天体の速度を測定するために必要な、静止している天体が放つ光の元の波長をどのようにして決定するのですか？',
  'role': 'user'},\
 {'content': '天文学者は、星に含まれる元素の独特なスペクトル指紋を利用します。これらの元素は、特定の既知の波長で光を放出および吸収し、吸収スペクトルを形成します。遠方の恒星から届く光を分析し、実験室で測定したこれらの元素のスペクトルと比較することで、天文学者はドップラー効果によるこれらの波長の変化を特定することができます。観測された変化から、光がどの程度赤方偏移または青方偏移しているかがわかり、それによって地球に対する視線方向の恒星の速度を計算することができます。
  'role': 'assistant'}]

そして、チャットテンプレートがこれらの会話をどのように変えたかを見てみましょう。

In [ ]:
dataset[5]["text"]

<|im_start|>user<|im_sep|>天文学者は、ドップラー効果を利用して天体の速度を測定するために必要な、静止している天体が放つ光の元の波長をどのようにして決定するのですか？<|im_end|>\
<|im_start|>assistant<|im_sep|>天文学者は、恒星に含まれる元素の固有のスペクトルフィンガープリントを利用します。これらの元素は、特定の既知の波長で光を放出および吸収し、吸収スペクトルを形成します。遠方の星から届く光を分析し、実験室で測定したこれらの元素のスペクトルと比較することで、天文学者はドップラー効果によるこれらの波長の変化を特定することができます。観測された変化から、光がどの程度赤方偏移または青方偏移したかがわかり、それによって地球に対する視線に沿った星の速度を計算することができます。<|im_end|>

### モデルをトレーニングする
では、Huggingface TRLの `SFTTrainer` を使ってみましょう！ 詳しい説明はこちら： [TRL SFT ドキュメント](https://huggingface.co/docs/trl/sft_trainer)。 \
処理を高速化するために 60 ステップを実行しますが、フル実行の場合は `num_train_epochs=1` を設定し、`max_steps=None` をオフにします。TRLの `DPOTrainer` もサポートしています！

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

また、Unslothの `train_on_completions` メソッドを使用して、アシスタントの出力のみを学習し、ユーザーの入力の損失を無視します。

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

マスキングが実際に実行されていることを確認します。

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"]) # input_ids は、テキストがトークン化され、モデルが理解できる数値IDに変換されたものでそれを元のテキストにデコードしています。

<|im_start|>user<|im_sep|>天文学者は、ドップラー効果を利用して天体の速度を測定するために必要な、静止している天体が放つ光の元の波長をどのようにして決定するのですか？<|im_end|>\
<|im_start|>assistant<|im_sep|>天文学者は、恒星に含まれる元素の固有のスペクトルフィンガープリントを利用します。これらの元素は、特定の既知の波長で光を放出および吸収し、吸収スペクトルを形成します。遠方の星から届く光を分析し、実験室で測定したこれらの元素のスペクトルと比較することで、天文学者はドップラー効果によるこれらの波長の変化を特定することができます。観測された変化から、光がどの程度赤方偏移または青方偏移したかがわかり、それによって地球に対する視線に沿った星の速度を計算することができます。<|im_end|>

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0] # 空白 (" ") をトークン化し、その input_idsの0番目を space 変数に格納します
print(space)
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]) # 5番目のサンプルのラベルデータを取得します。ラベルデータは、モデルが予測すべき出力（ここではアシスタントの発言）に対応します。 train_on_responses_only関数ではこのようにユーザーの発言をマスク（-100) で置き換えています。（spaceに置き換えて出力してみることで確認できました）

天文学者は、恒星に含まれる元素の独特なスペクトル指紋を利用しています。これらの元素は、特定の既知の波長で光を放出および吸収し、吸収スペクトルを形成します。遠方の星から届く光を分析し、実験室で測定したこれらの元素のスペクトルと比較することで、天文学者はドップラー効果によるこれらの波長の変化を特定することができます。観測された変化から、光がどの程度赤方偏移または青方偏移したかがわかり、それによって地球に対する視線に沿った星の速度を計算することができます。<|im_end|>

システムとインストラクションのプロンプトが正しくマスクされていることがわかります！

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3) # model, tokenizer, dataset, ライブラリが内部的に使用するメモリなど
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.967 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

991.7127 seconds used for training.
16.53 minutes used for training.
Peak reserved memory = 12.42 GB.
Peak reserved memory for training = 2.453 GB.
Peak reserved memory % of max memory = 84.255 %.
Peak reserved memory for training % of max memory = 16.641 %.


### 推論
モデルを実行してみましょう！ 命令と入力を変更できます。出力は空白のままにしてください！

**[NEW] Llama-3.1 8b Instructの無料Colabで2倍高速な推論を試してみましょう。[こちら](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

# アテンションマスクを作成
attention_mask = torch.ones_like(inputs)
attention_mask[inputs == tokenizer.pad_token_id] = 0

# モデルにアテンションマスクを渡す
outputs = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask.to("cuda"),
    max_new_tokens=64,
    use_cache=True,
    temperature=1.5,
    min_p=0.1,
)
tokenizer.batch_decode(outputs)

['<|im_start|>user<|im_sep|>Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|><|im_start|>assistant<|im_sep|>The next number in the Fibonacci sequence is 13. The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. In this case, the sequence starts with 1 and 1, and each subsequent number is the sum of the']

['<|im_start|>user<|im_sep|>フィボナッチ数列を続ける：1、1、2、3、5、8、<|im_end|>\
<|im_start|>assistant<|im_sep|>フィボナッチ数列の次の数は13です。\
フィボナッチ数列とは、各数が直前の2つの数の和となる一連の数字であり、通常は0と1から始まります。この場合、1と1から始まり、その後の各数は直前の2つの数の和となります。]

 また、`TextStreamer` を使用して連続して推論を行うこともできます。これにより、生成をトークンごとに確認できるため、全体を待つ必要がなくなります！

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

The next number in the Fibonacci sequence is 13. The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. In this case, the sequence starts with 1 and 1, and each subsequent number is the sum of the two preceding numbers. So, the next number after 8 is 13, which is the sum of 5 and 8.<|im_end|>


フィボナッチ数列の次の数は13です。フィボナッチ数列とは、通常0と1から始まり、各数が直前の2つの数の合計となる一連の数です。この場合、数列は1と1から始まり、その後の各数は直前の2つの数の合計です。したがって、8の次の数は13であり、これは5と8の合計です。<|im_end|>

### 調整したモデルの保存と読み込み
最終モデルをLoRAアダプターとして保存するには、Huggingfaceの `push_to_hub` を使用してオンラインで保存するか、`save_pretrained` を使用してローカルで保存します。

**[注意]** これはLoRAアダプターのみを保存し、完全なモデルは保存しません。16ビットまたはGGUFに保存するには、下にスクロールしてください！

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model/tokenizer.json')

推論用に保存したばかりのLoRAアダプターをロードしたい場合は、`False`を`True`に設定します。

In [ ]:
if False: # コードブロック全体の実行を無効化しています。
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

エッフェル塔は、フランスの首都パリにある高い塔です。高さは324メートル（1,063フィート）で、1889年に完成しました。この塔は、フランス革命100周年を祝う1889年の万国博覧会のために、ギュスターヴ・エッフェルとそのチームによって設計されました。錬鉄でできており、3つのレベルから構成され、観光客が訪れることができます。エッフェル塔は世界で最も有名なランドマークのひとつであり、毎年何百万人もの観光客が訪れます。<|im_end|>

Hugging FaceのAutoModelForPeftCausalLMも使用できます。unslothがインストールされていない場合のみ、これを使用してください。4bitモデルのダウンロードがサポートされていないため、処理が非常に遅くなる可能性があります。また、Unslothの推論は2倍高速です。(Peft: Parameter efficient fine-tuning)

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model/lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model/lora_model")

### VLLM用のfloat16への保存

また、`float16` への直接保存もサポートしています。float16 には `merged_16bit` を、int4 には `merged_4bit` を選択してください。また、`lora` アダプターをフォールバックとして使用することもできます。Hugging Face アカウントにアップロードするには、`push_to_hub_merged` を使用してください！ 個人用トークンは https://huggingface.co/settings/tokens から入手できます。

In [ ]:
from google.colab import userdata
# Merge to 16bit
if False: model.save_pretrained_merged("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp 変換
`GGUF` / `llama.cpp` に保存するには、現在ネイティブでサポートしています。`llama.cpp` をクローンし、デフォルトで `q8_0` に保存します。`q4_k_m` のようなすべてのメソッドを許可します。ローカル保存には `save_pretrained_gguf` を、HF へのアップロードには `push_to_hub_gguf` を使用します。

サポートされている量子化手法（全リストは [Docs](https://docs.unsloth.ai/basics/saving-and-using-models/saving-to-gguf) を参照）：
* `q8_0` - 高速変換。リソースの使用量は多いが、概ね許容できる。
* `q4_k_m` - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、それ以外にはQ4_Kを使用します。
* `q5_k_m` - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、それ以外にはQ5_Kを使用します。

[**NEW**] Ollamaへの微調整と自動エクスポートを行うには、[Ollamaノートブック](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)をお試しください。

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("/content/drive/MyDrive/Colab Notebooks/weights/Phi_4/model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

次に、`model-unsloth.gguf`ファイルまたは`model-unsloth-Q4_K_M.gguf`ファイルをllama.cppまたはJanやOpen WebUIのようなUIベースのシステムで使用します。Janは[こちら](https://github.com/janhq/jan)から、Open WebUIは[こちら](https://github.com/open-webui/open-webui)からインストールできます

Unslothについて何か質問があれば、[Discord](https://discord.gg/unsloth)チャンネルをご利用ください！バグを見つけた場合や、LLMの最新情報を入手したい場合、またはヘルプが必要な場合、プロジェクトに参加したい場合など、Discordにぜひご参加ください！

その他のリンク：
1. Llama 3.2 会話型ノートブック。 [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Ollamaへの微調整の保存。[フリーノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 ビジョン微調整 - 放射線画像のユースケース。 [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. DPO、ORPO、継続的な事前トレーニング、会話の微調整などについては、[ドキュメント](https://docs.unsloth.ai/get-started/unsloth-notebooks)のノートブックをご覧ください！

<div class=「align-center」>
  <a href=「https://unsloth.ai」><img src=「https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png」 width=「115」></a>
  <a href=「https://discord.gg/unsloth」><img src=「https://github.com/unslothai/unsloth/raw/main/images/Discord.png」 width=「145」></a>
  <a href=「https://docs.unsloth.ai/」><img src=「https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true」 width=「125」>
</div>

